In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Braindev'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['Class'].to_numpy()

In [ ]:
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20, compute_umap=True)

In [3]:
model_path_base = f'checkpoints/{dataset}_notime'
figure_path_base = f'figures/{dataset}_notime'
data_path = f'data/velovae/discrete/{dataset}_notime'
gene_plot = ['Mapt', 'Tmsb10', 'Fabp7', 'Npm1']

# Informative Time Prior

In [4]:
capture_time = adata.obs['Age'].to_numpy()
tprior = np.array([float(x[1:]) for x in capture_time])
adata.obs['tprior'] = tprior - tprior.min()

## Discrete VAE

In [5]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'


torch.manual_seed(2022)
np.random.seed(2022)

dvae = vv.model.DVAE(adata, 
                     tmax=20, 
                     dim_z=5, 
                     device='cuda:0',
                     init_method='tprior',
                     init_key='tprior',
                     tprior=None,
                     init_ton_zero=False)

dvae.train(adata, plot=True, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 5, 0
Initialization using prior time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Library scale (U): Max=10.47, Min=0.00, Mean=0.94
Library scale (S): Max=17.49, Min=0.16, Mean=1.00
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 164, test iteration: 326
Epoch 1: Train ELBO = -2578.790, Test ELBO = -19158.982, 	 Total Time =   0 h :  0 m : 12 s
Epoch 100: Train ELBO = -1289.482, Test ELBO = -1306.258, 	 Total Time =   0 h :  3 m : 20 s
Epoch 200: Train ELBO = -1256.141, Test ELBO = -1272.202, 	 Total Time =   0 h :  6 m : 29 s
*********       Stage 1: Early Stop Tri

  0%|          | 0/29948 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.000
Average Set Size: 1208
Epoch 250: Train ELBO = -1297.613, Test ELBO = -1680.720, 	 Total Time =   0 h :  8 m : 45 s
*********       Stage 2: Early Stop Triggered at epoch 265.       *********
*********              Finished. Total Time =   0 h : 10 m : 59 s             *********
Final: Train ELBO = -1284.454,           Test ELBO = -1287.634
       Training MSE = 2.804, Test MSE = 2.664


# Discrete FullVB

In [6]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
dfullvb = vv.model.DVAEFullVB(adata, 
                              tmax=20, 
                              dim_z=5, 
                              device='cuda:0',
                              init_method="tprior",
                              init_key="tprior",
                              tprior=None,
                              init_ton_zero=False)

dfullvb.train(adata, plot=True, gene_plot=gene_plot, figure_path=figure_path)

dfullvb.save_model(model_path, 'encoder', 'decoder')
dfullvb.save_anndata(adata, 'dfullvb', data_path, f'{dataset}.h5ad')

Detecting zero scaling factors: 5, 0
Initialization using prior time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Library scale (U): Max=10.47, Min=0.00, Mean=0.94
Library scale (S): Max=17.49, Min=0.16, Mean=1.00
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 164, test iteration: 326
Epoch 1: Train ELBO = -2595.597, Test ELBO = -19268.738, 	 Total Time =   0 h :  0 m :  9 s
*********       Stage 1: Early Stop Triggered at epoch 44.       *********
*********                      Stage  2                       *********


  0%|          | 0/29948 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.000
Average Set Size: 2203
Epoch 45: Train ELBO = -1404.056, Test ELBO = -5654.775, 	 Total Time =   0 h :  2 m : 42 s
*********       Stage 2: Early Stop Triggered at epoch 60.       *********
*********              Finished. Total Time =   0 h :  5 m :  0 s             *********
Final: Train ELBO = -1390.051,           Test ELBO = -1395.322
       Training MSE = 3.732, Test MSE = 3.482


# Evaluation

In [ ]:
cluster_edges = [('Neural tube','Radial glia'),
                 ('Radial glia', 'Neuroblast'),
                 ('Radial glia', 'Glioblast'),
                 ('Radial glia', 'Oligodendrocyte'),
                 ('Radial glia', 'Ependymal'),
                 ('Neural crest', 'Mesenchyme'),
                 ('Mesenchyme','Fibroblast')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)